In [1]:
from nipype.pipeline import engine as pe
from nipype.interfaces import utility as niu

In [2]:
dwi_file = '/projects/mjoseph/pipelines/testing/data/sub-0880002/ses-01/dwi/sub-0880002_ses-01_acq-singleshell19dir_dwi.nii.gz'

In [33]:
def init_test_wf():
    wf = pe.Workflow(name="test")

    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file"]), name="inputnode")

    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")

    denoise = pe.Node(DWIDenoise(), name="denoise")
    
    unring = pe.Node(MRDeGibbs(), name="unring")

    wf.connect([(inputnode, denoise, [("dwi_file", "in_file")]),
                (denoise, unring, [("out_file", "in_file")]),
                (unring, outputnode, [("out_file", "out_file")])])
    
    return wf

In [34]:
test_wf = init_test_wf()
inputspec = test_wf.get_node("inputnode")
inputspec.inputs.dwi_file = dwi_file
test_wf.run()

190627-17:38:59,948 nipype.workflow INFO:
	 Workflow test settings: ['check', 'execution', 'logging', 'monitoring']
190627-17:39:00,35 nipype.workflow INFO:
	 Running serially.
190627-17:39:00,37 nipype.workflow INFO:
	 [Node] Setting-up "test.denoise" in "/tmp/tmpbinecjkb/test/denoise".
190627-17:39:00,42 nipype.workflow INFO:
	 [Node] Running "denoise" ("__main__.DWIDenoise"), a CommandLine Interface with command:
dwidenoise -noise sub-0880002_ses-01_acq-singleshell19dir_dwi_noise.nii.gz /projects/mjoseph/pipelines/testing/data/sub-0880002/ses-01/dwi/sub-0880002_ses-01_acq-singleshell19dir_dwi.nii.gz sub-0880002_ses-01_acq-singleshell19dir_dwi_denoised.nii.gz
190627-17:39:00,529 nipype.interface INFO:
dwidenoise: [100%] uncompressing image "/projects/mjoseph/pipelines/testing/data/sub-0880002/ses-01/dwi/sub-0880002_ses-01_acq-singleshell19dir_dwi.nii.gz"
190627-17:39:00,823 nipype.interface INFO:
dwidenoise: [100%] preloading data for "/projects/mjoseph/pipelines/testing/data/sub-088

In [29]:
from __future__ import (print_function, division, unicode_literals,
                        absolute_import)

import os

from nipype.interfaces.base import (CommandLineInputSpec, CommandLine, traits, TraitedSpec,
                    File, isdefined, Undefined, InputMultiObject)
from nipype.interfaces.mrtrix3.base import MRTrix3BaseInputSpec, MRTrix3Base


class DWIDenoiseInputSpec(MRTrix3BaseInputSpec):
    in_file = File(
        exists=True,
        argstr='%s',
        position=-2,
        mandatory=True,
        desc='input DWI image')
    mask = File(
        exists=True,
        argstr='-mask %s',
        position=1,
        desc='mask image')
    extent = traits.Tuple((traits.Int, traits.Int, traits.Int),
        argstr='-extent %d,%d,%d',
        desc='set the window size of the denoising filter. (default = 5,5,5)')
    noise = File(
        argstr='-noise %s',
        name_template='%s_noise',
        name_source=['in_file'],
        keep_extension=True,
        desc='the output noise map')
    out_file = File(
        argstr='%s',
        name_template='%s_denoised',
        name_source=['in_file'],
        keep_extension=True,
        position=-1,
        desc='the output denoised DWI image')

class DWIDenoiseOutputSpec(TraitedSpec):
    out_file = File(desc='the output denoised DWI image', exists=True)
    noise = File(desc='the output noise map (if generated)', exists=True)

class DWIDenoise(MRTrix3Base):
    """
    Denoise DWI data and estimate the noise level based on the optimal
    threshold for PCA.
    DWI data denoising and noise map estimation by exploiting data redundancy
    in the PCA domain using the prior knowledge that the eigenspectrum of
    random covariance matrices is described by the universal Marchenko Pastur
    distribution.
    Important note: image denoising must be performed as the first step of the
    image processing pipeline. The routine will fail if interpolation or
    smoothing has been applied to the data prior to denoising.
    Note that this function does not correct for non-Gaussian noise biases.
    For more information, see
    <https://mrtrix.readthedocs.io/en/latest/reference/commands/dwidenoise.html>
    Example
    -------
    >>> import nipype.interfaces.mrtrix3 as mrt
    >>> denoise = mrt.DWIDenoise()
    >>> denoise.inputs.in_file = 'dwi.mif'
    >>> denoise.inputs.mask = 'mask.mif'
    >>> denoise.cmdline                               # doctest: +ELLIPSIS
    'dwidenoise -mask mask.mif dwi.mif dwi_denoised.mif'
    >>> denoise.run()                                 # doctest: +SKIP
    """

    _cmd = 'dwidenoise'
    input_spec = DWIDenoiseInputSpec
    output_spec = DWIDenoiseOutputSpec

In [32]:
class MRDeGibbsInputSpec(MRTrix3BaseInputSpec):
    in_file = File(
        exists=True,
        argstr='%s',
        position=-2,
        mandatory=True,
        desc='input DWI image')
    axes = traits.ListInt(
        default_value=[0,1],
        usedefault=True,
        sep=',',
        minlen=2,
        maxlen=2,
        argstr='-axes %s',
        desc='indicate the plane in which the data was acquired (axial = 0,1; '
             'coronal = 0,2; sagittal = 1,2')
    nshifts = traits.Int(
        default_value=20,
        usedefault=True,
        argstr='-nshifts %d',
        desc='discretization of subpixel spacing (default = 20)')
    minW = traits.Int(
        default_value=1,
        usedefault=True,
        argstr='-minW %d',
        desc='left border of window used for total variation (TV) computation '
             '(default = 1)')
    maxW = traits.Int(
        default_value=3,
        usedefault=True,
        argstr='-maxW %d',
        desc='right border of window used for total variation (TV) computation '
             '(default = 3)')
    out_file = File(name_template='%s_unr',
        name_source='in_file',
        keep_extension=True,
        argstr='%s',
        position=-1,
        desc='the output unringed DWI image',
        genfile=True)

class MRDeGibbsOutputSpec(TraitedSpec):
    out_file = File(desc='the output unringed DWI image', exists=True)

class MRDeGibbs(MRTrix3Base):
    """
    Remove Gibbs ringing artifacts.
    This application attempts to remove Gibbs ringing artefacts from MRI images
    using the method of local subvoxel-shifts proposed by Kellner et al.
    This command is designed to run on data directly after it has been
    reconstructed by the scanner, before any interpolation of any kind has
    taken place. You should not run this command after any form of motion
    correction (e.g. not after dwipreproc). Similarly, if you intend running
    dwidenoise, you should run this command afterwards, since it has the
    potential to alter the noise structure, which would impact on dwidenoise's
    performance.
    Note that this method is designed to work on images acquired with full
    k-space coverage. Running this method on partial Fourier ('half-scan') data
    may lead to suboptimal and/or biased results, as noted in the original
    reference below. There is currently no means of dealing with this; users
    should exercise caution when using this method on partial Fourier data, and
    inspect its output for any obvious artefacts.
    For more information, see
    <https://mrtrix.readthedocs.io/en/latest/reference/commands/mrdegibbs.html>
    Example
    -------
    >>> import nipype.interfaces.mrtrix3 as mrt
    >>> unring = mrt.MRDeGibbs()
    >>> unring.inputs.in_file = 'dwi.mif'
    >>> unring.cmdline
    'mrdegibbs -axes 0,1 -maxW 3 -minW 1 -nshifts 20 dwi.mif dwi_unr.mif'
    >>> unring.run()                                 # doctest: +SKIP
    """

    _cmd = 'mrdegibbs'
    input_spec = MRDeGibbsInputSpec
    output_spec = MRDeGibbsOutputSpec

In [ ]:
    def _gen_outfilename(self):
        out_file = self.inputs.out_file
        if not isdefined(out_file) and isdefined(self.inputs.in_file):
            out_file = self._gen_fname(self.inputs.in_file, suffix='_denoised')
        return os.path.abspath(out_file)
    
    def _gen_fname(self,
               basename,
               cwd=None,
               suffix=None,
               change_ext=True,
               ext=None):

        if basename == '':
            msg = 'Unable to generate filename for command %s. ' % self.cmd
            msg += 'basename is not set!'
            raise ValueError(msg)
        if cwd is None:
            cwd = os.getcwd()
        if ext is None:
            ext = Info.output_type_to_ext(self.inputs.output_type)
        if change_ext:
            if suffix:
                suffix = ''.join((suffix, ext))
            else:
                suffix = ext
        if suffix is None:
            suffix = ''
        fname = fname_presuffix(
            basename, suffix=suffix, use_ext=False, newpath=cwd)
        return fname

    def _list_outputs(self):
        outputs = self.output_spec().get()
        outputs['out_file'] = self._gen_outfilename()
        if (isdefined(self.inputs.noise) and self.inputs.noise):
            outputs['noise_file'] = self._gen_fname(outputs['out_file'], suffix='_noise')
        return outputs